In [13]:
import pandas as pd
import numpy as np
import os
import cv2
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import tensorflow as tf
import tensorflow.keras as k
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, AveragePooling2D, Flatten, Dropout


In [14]:
path_folder = "/kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train"
class_name = os.listdir(path_folder)
class_name.sort()
class_name = class_name[:4] + class_name[20:23]
print(class_name)
image_data = []
label_data = []
count = 0
for folder in class_name:
    images = os.listdir(path_folder + "/" + folder)
    print("Loading Folder -- {} " .format(folder), "The Count of Classes ==> ",count)
    for img in images:
        image = cv2.imread(path_folder + "/" + folder + "/" + img)
        image = cv2.resize(image, (100, 100))
        image_data.append(image)
        label_data.append(count)
    count += 1
print("---- Done ----------- ")

['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy']
Loading Folder -- Apple___Apple_scab  The Count of Classes ==>  0
Loading Folder -- Apple___Black_rot  The Count of Classes ==>  1
Loading Folder -- Apple___Cedar_apple_rust  The Count of Classes ==>  2
Loading Folder -- Apple___healthy  The Count of Classes ==>  3
Loading Folder -- Potato___Early_blight  The Count of Classes ==>  4
Loading Folder -- Potato___Late_blight  The Count of Classes ==>  5
Loading Folder -- Potato___healthy  The Count of Classes ==>  6
---- Done ----------- 


In [15]:
data = np.array(image_data)
data = data.astype("float32")
data = data/255.0

label = np.array(label_data)

In [16]:
print(data.shape)

(13473, 100, 100, 3)


In [17]:
label_num = to_categorical(label, len(class_name))
label_num[100]

array([1., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [18]:
x_img, y_img = shuffle(data, label_num)
x_train, x_test, y_train, y_test = train_test_split(x_img, y_img, train_size=0.8)

In [19]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((10778, 100, 100, 3), (10778, 7), (2695, 100, 100, 3), (2695, 7))

In [20]:
model = k.models.Sequential()

model.add(k.layers.Conv2D(16, (5, 5), activation="relu", input_shape=(100, 100, 3), padding="same"))
model.add(k.layers.AveragePooling2D((2, 2)))

model.add(k.layers.Conv2D(32, (4, 4), activation="relu",  padding="same"))
model.add(k.layers.AveragePooling2D((2, 2)))


model.add(k.layers.Conv2D(64, (3, 3), activation="relu", padding="same"))
model.add(k.layers.AveragePooling2D((2, 2)))

model.add(k.layers.Conv2D(128, (2, 2), activation="relu", padding="same"))
model.add(k.layers.MaxPool2D((2, 2)))
model.add(k.layers.Flatten())

model.add(k.layers.Dense(256, activation="relu"))
model.add(k.layers.Dropout(0.5))

model.add(k.layers.Dense(32, activation="relu"))
model.add(k.layers.Dropout(0.2))

model.add(k.layers.Dense(7, activation="softmax"))

model.compile(optimizer="adam", loss=k.losses.CategoricalCrossentropy(), metrics=["accuracy"])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 100, 100, 16)      1216      
                                                                 
 average_pooling2d (AverageP  (None, 50, 50, 16)       0         
 ooling2D)                                                       
                                                                 
 conv2d_1 (Conv2D)           (None, 50, 50, 32)        8224      
                                                                 
 average_pooling2d_1 (Averag  (None, 25, 25, 32)       0         
 ePooling2D)                                                     
                                                                 
 conv2d_2 (Conv2D)           (None, 25, 25, 64)        18496     
                                                                 
 average_pooling2d_2 (Averag  (None, 12, 12, 64)       0

In [21]:
history = model.fit(x_train, y_train, epochs=50, validation_data=(x_test, y_test), validation_split=0.4)

Epoch 1/50
337/337 [==============================] - 14s 13ms/step - loss: 1.4616 - accuracy: 0.4224 - val_loss: 1.0066 - val_accuracy: 0.6404
Epoch 2/50
337/337 [==============================] - 3s 10ms/step - loss: 0.9619 - accuracy: 0.6554 - val_loss: 0.8566 - val_accuracy: 0.7132
Epoch 3/50
337/337 [==============================] - 3s 10ms/step - loss: 0.6699 - accuracy: 0.7670 - val_loss: 0.4176 - val_accuracy: 0.8575
Epoch 4/50
337/337 [==============================] - 4s 11ms/step - loss: 0.4896 - accuracy: 0.8400 - val_loss: 0.8702 - val_accuracy: 0.7291
Epoch 5/50
337/337 [==============================] - 3s 10ms/step - loss: 0.4350 - accuracy: 0.8589 - val_loss: 0.4555 - val_accuracy: 0.8538
Epoch 6/50
337/337 [==============================] - 3s 9ms/step - loss: 0.3458 - accuracy: 0.8888 - val_loss: 0.3389 - val_accuracy: 0.8905
Epoch 7/50
337/337 [==============================] - 3s 10ms/step - loss: 0.2994 - accuracy: 0.9081 - val_loss: 0.6430 - val_accuracy: 0.8033

In [22]:
loss, accuracy = model.evaluate(x_test, y_test)

print(f"The Accuracy Testing Data is ==> {accuracy} %")
print(f"The loss Testing Data is ==> {loss}")


85/85 [==============================] - 0s 5ms/step - loss: 0.1512 - accuracy: 0.9636
The Accuracy Testing Data is ==> 0.9636363387107849 %
The loss Testing Data is ==> 0.1511993706226349


In [23]:
from tensorflow.keras.models import save_model
model.save("trained_model.h5")